In [1]:
from db_communication import db_communication
import numpy as np
import pandas as pd
from datetime import datetime

### Init tables/ dataframes

In [2]:
config = {
    'user': 'root',
    'password': 'FDS-apm1',
    'host': 'min-ifm-xdm.ad.fh-bielefeld.de',
    'port': '3306',
    'database': 'Fahrradshop' 
}
my_db = db_communication(config)

Successfully connected.


In [3]:
cluster_df = my_db.get_table('Merkmalcluster')
parts_df = my_db.get_table('Einzelteile')
features_df = my_db.get_table('Merkmale')

config_df = my_db.get_table('Konfiguration')
config_df.index = pd.MultiIndex.from_arrays([config_df['AuftragNr'],config_df['KonfigNr']])
config_df = config_df['MerkmalNr'].to_frame()

assemblyGroup_df = my_db.get_table("Arbeitsschrittgruppe")

In [5]:
# read direct demand matrix
filename = 'DirectDemandMatrix_20200623_165446.xlsx'
directDemand_df = pd.read_excel(filename, header=[0,1], index_col=0)
directDemand_df

Cluster                                          Rahmen                      \
Merkmal                                 Herren Citybike Herren Trekkingbike   
Einzelteil                                                                    
Surly Long Haul Trucker Rahmenkit                   1.0                 NaN   
Surly Troll Rahmenkit                               NaN                 1.0   
Salsa Cutthroat Rahmenkit                           NaN                 NaN   
VOTEC VRC Framekit                                  NaN                 NaN   
Cervelo C3 Frameset                                 NaN                 NaN   
...                                                 ...                 ...   
Xpedo Detox Pedale                                  NaN                 NaN   
NOW8 M36 Flat Pedals 6 Pins                         NaN                 NaN   
Moto Reflex Pedale                                  NaN                 NaN   
Time ATAC XC6 X-Country Pedale                      NaN                 NaN   
Shimano Dura Ace PD-R9100 Pedale carbon             NaN                 NaN   

Cluster                                                         \
Merkmal                                 Herren Trekking Carbon   
Einzelteil                                                       
Surly Long Haul Trucker Rahmenkit                          NaN   
Surly Troll Rahmenkit                                      NaN   
Salsa Cutthroat Rahmenkit                                  1.0   
VOTEC VRC Framekit                                         NaN   
Cervelo C3 Frameset                                        NaN   
...                                                        ...   
Xpedo Detox Pedale                                         NaN   
NOW8 M36 Flat Pedals 6 Pins                                NaN   
Moto Reflex Pedale                                         NaN   
Time ATAC XC6 X-Country Pedale                             NaN   
Shimano Dura Ace PD-R9100 Pedale carbon                    NaN   

Cluster                                                           \
Merkmal                                 Unisex Rennrad Aluminium   
Einzelteil                                                         
Surly Long Haul Trucker Rahmenkit                            NaN   
Surly Troll Rahmenkit                                        NaN   
Salsa Cutthroat Rahmenkit                                    NaN   
VOTEC VRC Framekit                                           1.0   
Cervelo C3 Frameset                                          NaN   
...                                                          ...   
Xpedo Detox Pedale                                           NaN   
NOW8 M36 Flat Pedals 6 Pins                                  NaN   
Moto Reflex Pedale                                           NaN   
Time ATAC XC6 X-Country Pedale                               NaN   
Shimano Dura Ace PD-R9100 Pedale carbon                      NaN   

Cluster                                                                       \
Merkmal                                 Unisex Rennrad Carbon Damen Citybike   
Einzelteil                                                                     
Surly Long Haul Trucker Rahmenkit                         NaN            NaN   
Surly Troll Rahmenkit                                     NaN            NaN   
Salsa Cutthroat Rahmenkit                                 NaN            NaN   
VOTEC VRC Framekit                                        NaN            NaN   
Cervelo C3 Frameset                                       1.0            NaN   
...                                                       ...            ...   
Xpedo Detox Pedale                                        NaN            NaN   
NOW8 M36 Flat Pedals 6 Pins                               NaN            NaN   
Moto Reflex Pedale                                        NaN            NaN   
Time ATAC XC6 X-Country Pedale                         

### Preparation for assembly steps

In [6]:
# prepare assembly dataframe by direct demand matrix
df_assembly = pd.DataFrame(columns=['Merkmal','Einzelteil','Anzahl','Arbeitsschrittgruppe','Montagezeit'])

#iterate through each separate part
for row_name,row in directDemand_df.iterrows():
    row = row.dropna().to_frame()
    # iterate through each feature per part
    for _,sub_row in row.iterrows():
        # get data
        feature_name = sub_row.to_frame().columns[0][1]
        part_name = sub_row.index.values[0]
        part_count = sub_row.values[0]
        # append sub_row
        df_tmp = pd.DataFrame(columns=['Merkmal','Einzelteil','Anzahl'],data=[[feature_name,part_name,part_count]])
        df_assembly = df_assembly.append(df_tmp)
    
# SchrittNr as index
df_assembly.index = np.arange(1,len(df_assembly)+1)
df_assembly.index.name = 'SchrittNr'
df_assembly

,Merkmal,Einzelteil,Anzahl,Arbeitsschrittgruppe,Montagezeit
SchrittNr,,,,,
1,Herren Citybike,Surly Long Haul Trucker Rahmenkit,1.0,NaN,NaN
2,Herren Trekkingbike,Surly Troll Rahmenkit,1.0,NaN,NaN
3,Herren Trekking Carbon,Salsa Cutthroat Rahmenkit,1.0,NaN,NaN
4,Unisex Rennrad Aluminium,VOTEC VRC Framekit,1.0,NaN,NaN
5,Unisex Rennrad Carbon,Cervelo C3 Frameset,1.0,NaN,NaN
...,...,...,...,...,...
214,Trekking Pedale,Xpedo Detox Pedale,2.0,NaN,NaN
215,Sport Pedale,NOW8 M36 Flat Pedals 6 Pins,2.0,NaN,NaN
216,Komfort Pedale,Moto Reflex Pedale,2.0,NaN,NaN


In [7]:
# save assemply dataframe as Excel with current datetime as version
# in order to add 'expert knowledge' in 'Arbeitsschrittgruppe' and 'Montagezeit'
df_assembly.to_excel('AssemblyTable_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx')

__HINT:__ <br> 
After saving the 'df_assembly' it is necessary to connect the 'Arbeitsschrittgruppe'-column with the individual parts that are to be assembled in a node in the algorithm of the assembly priority graph

### Read assembly table

In [8]:
filename = 'AssemblyTable_20200623_170338.xlsx'
df_assembly_new = pd.read_excel(filename, header=[0], index_col='SchrittNr')
df_assembly_new

,Merkmal,Einzelteil,Anzahl,Arbeitsschrittgruppe,Montagezeit
SchrittNr,,,,,
1,Herren Citybike,Surly Long Haul Trucker Rahmenkit,1,1,30
2,Herren Trekkingbike,Surly Troll Rahmenkit,1,1,30
3,Herren Trekking Carbon,Salsa Cutthroat Rahmenkit,1,1,30
4,Unisex Rennrad Aluminium,VOTEC VRC Framekit,1,1,30
5,Unisex Rennrad Carbon,Cervelo C3 Frameset,1,1,30
...,...,...,...,...,...
214,Trekking Pedale,Xpedo Detox Pedale,2,15,20
215,Sport Pedale,NOW8 M36 Flat Pedals 6 Pins,2,15,20
216,Komfort Pedale,Moto Reflex Pedale,2,15,20


### Manipulate assembly table in order match the schema of the database

In [9]:
# copy cells where the part belongs to more than one working group step and delete the olds cells

old_index_name = df_assembly_new.index.name #remind index name
del_lst = []
for idx, row in df_assembly_new.iterrows():
    #print(idx, row['Arbeitsschrittgruppe'])
    # check for multiple working group steps
    if type(row['Arbeitsschrittgruppe']) != int:
        row_lst = np.array(row['Arbeitsschrittgruppe'].split(';')).astype(int)
        
        # for each multiple 
        for element in row_lst:
            row_append = pd.Series(row)
            row_append['Arbeitsschrittgruppe'] = element
            
            if row_append['Anzahl'] > 1:
                row_append['Anzahl'] = int(row_append['Anzahl'] / len(row_lst))
                
            df_assembly_new = df_assembly_new.append(row_append,ignore_index=True)
        
        # save idx for a later drop
        del_lst.append(idx-1)
df_assembly_new = df_assembly_new.drop(del_lst)
# reset index numbers
df_assembly_new = df_assembly_new.set_index(np.arange(1,len(df_assembly_new)+1))
df_assembly_new.index.name = old_index_name

In [10]:
df_assembly_new

,Merkmal,Einzelteil,Anzahl,Arbeitsschrittgruppe,Montagezeit
SchrittNr,,,,,
1,Herren Citybike,Surly Long Haul Trucker Rahmenkit,1,1,30
2,Herren Trekkingbike,Surly Troll Rahmenkit,1,1,30
3,Herren Trekking Carbon,Salsa Cutthroat Rahmenkit,1,1,30
4,Unisex Rennrad Aluminium,VOTEC VRC Framekit,1,1,30
5,Unisex Rennrad Carbon,Cervelo C3 Frameset,1,1,30
...,...,...,...,...,...
253,Bereifung Sport,SCHWALBE No.13 Fahrradschlauch,1,13,10
254,Bereifung Rennrad Standard,SCHWALBE No.15 Schlauch Rennrad,1,12,10
255,Bereifung Rennrad Standard,SCHWALBE No.15 Schlauch Rennrad,1,13,10


In [11]:
# mapping name to number
# features
feature_keys = features_df['Bezeichnung'].values
feature_values = features_df['MerkmalNr'].values
feature_mapping = dict(zip(feature_keys,feature_values))
# parts
part_keys = parts_df['Bezeichnung'].values
part_values = parts_df['EinzelteilNr'].values
part_mapping = dict(zip(part_keys,part_values))
# column names
col_mapping = {'Merkmal' : 'MerkmalNr', 
               'Einzelteil' : 'EinzelteilNr', 
               'Anzahl' : 'Anzahl Einzelteil',
               'Arbeitsschrittgruppe' : 'ArbeitsschrittgruppeNr'}

In [12]:
# apply mapping
df_assembly_new['Merkmal'] = df_assembly_new['Merkmal'].replace(feature_mapping)
df_assembly_new['Einzelteil'] = df_assembly_new['Einzelteil'].replace(part_mapping)
df_assembly_new = df_assembly_new.rename(columns=col_mapping)

In [13]:
df_assembly_new

,MerkmalNr,EinzelteilNr,Anzahl Einzelteil,ArbeitsschrittgruppeNr,Montagezeit
SchrittNr,,,,,
1,100001,1001,1,1,30
2,100002,1002,1,1,30
3,100003,1003,1,1,30
4,100004,1004,1,1,30
5,100005,1005,1,1,30
...,...,...,...,...,...
253,600006,6012,1,13,10
254,600007,6013,1,12,10
255,600007,6013,1,13,10


### Writting into the database

In [14]:
for idx,row in df_assembly_new.reset_index().iterrows():
    my_db.insert_data('Arbeitsschritt',row.to_dict())

INSERT INTO `Arbeitsschritt` (`SchrittNr`, `MerkmalNr`, `EinzelteilNr`, `Anzahl Einzelteil`, `ArbeitsschrittgruppeNr`, `Montagezeit`) VALUES ('1', '100001', '1001', '1', '1', '30');
INSERT INTO `Arbeitsschritt` (`SchrittNr`, `MerkmalNr`, `EinzelteilNr`, `Anzahl Einzelteil`, `ArbeitsschrittgruppeNr`, `Montagezeit`) VALUES ('2', '100002', '1002', '1', '1', '30');
INSERT INTO `Arbeitsschritt` (`SchrittNr`, `MerkmalNr`, `EinzelteilNr`, `Anzahl Einzelteil`, `ArbeitsschrittgruppeNr`, `Montagezeit`) VALUES ('3', '100003', '1003', '1', '1', '30');
INSERT INTO `Arbeitsschritt` (`SchrittNr`, `MerkmalNr`, `EinzelteilNr`, `Anzahl Einzelteil`, `ArbeitsschrittgruppeNr`, `Montagezeit`) VALUES ('4', '100004', '1004', '1', '1', '30');
INSERT INTO `Arbeitsschritt` (`SchrittNr`, `MerkmalNr`, `EinzelteilNr`, `Anzahl Einzelteil`, `ArbeitsschrittgruppeNr`, `Montagezeit`) VALUES ('5', '100005', '1005', '1', '1', '30');
INSERT INTO `Arbeitsschritt` (`SchrittNr`, `MerkmalNr`, `EinzelteilNr`, `Anzahl Einzelteil

INSERT INTO `Arbeitsschritt` (`SchrittNr`, `MerkmalNr`, `EinzelteilNr`, `Anzahl Einzelteil`, `ArbeitsschrittgruppeNr`, `Montagezeit`) VALUES ('46', '300004', '3010', '1', '4', '30');
INSERT INTO `Arbeitsschritt` (`SchrittNr`, `MerkmalNr`, `EinzelteilNr`, `Anzahl Einzelteil`, `ArbeitsschrittgruppeNr`, `Montagezeit`) VALUES ('47', '300005', '3010', '1', '4', '30');
INSERT INTO `Arbeitsschritt` (`SchrittNr`, `MerkmalNr`, `EinzelteilNr`, `Anzahl Einzelteil`, `ArbeitsschrittgruppeNr`, `Montagezeit`) VALUES ('48', '300006', '3010', '1', '4', '30');
INSERT INTO `Arbeitsschritt` (`SchrittNr`, `MerkmalNr`, `EinzelteilNr`, `Anzahl Einzelteil`, `ArbeitsschrittgruppeNr`, `Montagezeit`) VALUES ('49', '300007', '3010', '1', '4', '30');
INSERT INTO `Arbeitsschritt` (`SchrittNr`, `MerkmalNr`, `EinzelteilNr`, `Anzahl Einzelteil`, `ArbeitsschrittgruppeNr`, `Montagezeit`) VALUES ('50', '300004', '3011', '1', '4', '30');
INSERT INTO `Arbeitsschritt` (`SchrittNr`, `MerkmalNr`, `EinzelteilNr`, `Anzahl Einze

INSERT INTO `Arbeitsschritt` (`SchrittNr`, `MerkmalNr`, `EinzelteilNr`, `Anzahl Einzelteil`, `ArbeitsschrittgruppeNr`, `Montagezeit`) VALUES ('91', '700001', '7006', '1', '9', '30');
INSERT INTO `Arbeitsschritt` (`SchrittNr`, `MerkmalNr`, `EinzelteilNr`, `Anzahl Einzelteil`, `ArbeitsschrittgruppeNr`, `Montagezeit`) VALUES ('92', '700002', '7007', '1', '9', '30');
INSERT INTO `Arbeitsschritt` (`SchrittNr`, `MerkmalNr`, `EinzelteilNr`, `Anzahl Einzelteil`, `ArbeitsschrittgruppeNr`, `Montagezeit`) VALUES ('93', '700002', '7008', '1', '14', '30');
INSERT INTO `Arbeitsschritt` (`SchrittNr`, `MerkmalNr`, `EinzelteilNr`, `Anzahl Einzelteil`, `ArbeitsschrittgruppeNr`, `Montagezeit`) VALUES ('94', '700002', '7009', '1', '21', '30');
INSERT INTO `Arbeitsschritt` (`SchrittNr`, `MerkmalNr`, `EinzelteilNr`, `Anzahl Einzelteil`, `ArbeitsschrittgruppeNr`, `Montagezeit`) VALUES ('95', '700002', '7010', '1', '24', '30');
INSERT INTO `Arbeitsschritt` (`SchrittNr`, `MerkmalNr`, `EinzelteilNr`, `Anzahl Ei

INSERT INTO `Arbeitsschritt` (`SchrittNr`, `MerkmalNr`, `EinzelteilNr`, `Anzahl Einzelteil`, `ArbeitsschrittgruppeNr`, `Montagezeit`) VALUES ('137', '900001', '9003', '2', '20', '30');
INSERT INTO `Arbeitsschritt` (`SchrittNr`, `MerkmalNr`, `EinzelteilNr`, `Anzahl Einzelteil`, `ArbeitsschrittgruppeNr`, `Montagezeit`) VALUES ('138', '900002', '9003', '2', '20', '30');
INSERT INTO `Arbeitsschritt` (`SchrittNr`, `MerkmalNr`, `EinzelteilNr`, `Anzahl Einzelteil`, `ArbeitsschrittgruppeNr`, `Montagezeit`) VALUES ('139', '900003', '9003', '1', '20', '30');
INSERT INTO `Arbeitsschritt` (`SchrittNr`, `MerkmalNr`, `EinzelteilNr`, `Anzahl Einzelteil`, `ArbeitsschrittgruppeNr`, `Montagezeit`) VALUES ('140', '900001', '9004', '4', '20', '30');
INSERT INTO `Arbeitsschritt` (`SchrittNr`, `MerkmalNr`, `EinzelteilNr`, `Anzahl Einzelteil`, `ArbeitsschrittgruppeNr`, `Montagezeit`) VALUES ('141', '900002', '9004', '4', '20', '30');
INSERT INTO `Arbeitsschritt` (`SchrittNr`, `MerkmalNr`, `EinzelteilNr`, `An

INSERT INTO `Arbeitsschritt` (`SchrittNr`, `MerkmalNr`, `EinzelteilNr`, `Anzahl Einzelteil`, `ArbeitsschrittgruppeNr`, `Montagezeit`) VALUES ('182', '100007', '1019', '1', '17', '20');
INSERT INTO `Arbeitsschritt` (`SchrittNr`, `MerkmalNr`, `EinzelteilNr`, `Anzahl Einzelteil`, `ArbeitsschrittgruppeNr`, `Montagezeit`) VALUES ('183', '100007', '1019', '1', '18', '20');
INSERT INTO `Arbeitsschritt` (`SchrittNr`, `MerkmalNr`, `EinzelteilNr`, `Anzahl Einzelteil`, `ArbeitsschrittgruppeNr`, `Montagezeit`) VALUES ('184', '100001', '1020', '1', '17', '20');
INSERT INTO `Arbeitsschritt` (`SchrittNr`, `MerkmalNr`, `EinzelteilNr`, `Anzahl Einzelteil`, `ArbeitsschrittgruppeNr`, `Montagezeit`) VALUES ('185', '100001', '1020', '1', '18', '20');
INSERT INTO `Arbeitsschritt` (`SchrittNr`, `MerkmalNr`, `EinzelteilNr`, `Anzahl Einzelteil`, `ArbeitsschrittgruppeNr`, `Montagezeit`) VALUES ('186', '100006', '1020', '1', '17', '20');
INSERT INTO `Arbeitsschritt` (`SchrittNr`, `MerkmalNr`, `EinzelteilNr`, `An

INSERT INTO `Arbeitsschritt` (`SchrittNr`, `MerkmalNr`, `EinzelteilNr`, `Anzahl Einzelteil`, `ArbeitsschrittgruppeNr`, `Montagezeit`) VALUES ('227', '600001', '6009', '1', '13', '30');
INSERT INTO `Arbeitsschritt` (`SchrittNr`, `MerkmalNr`, `EinzelteilNr`, `Anzahl Einzelteil`, `ArbeitsschrittgruppeNr`, `Montagezeit`) VALUES ('228', '600002', '6009', '1', '12', '30');
INSERT INTO `Arbeitsschritt` (`SchrittNr`, `MerkmalNr`, `EinzelteilNr`, `Anzahl Einzelteil`, `ArbeitsschrittgruppeNr`, `Montagezeit`) VALUES ('229', '600002', '6009', '1', '13', '30');
INSERT INTO `Arbeitsschritt` (`SchrittNr`, `MerkmalNr`, `EinzelteilNr`, `Anzahl Einzelteil`, `ArbeitsschrittgruppeNr`, `Montagezeit`) VALUES ('230', '600003', '6009', '1', '12', '20');
INSERT INTO `Arbeitsschritt` (`SchrittNr`, `MerkmalNr`, `EinzelteilNr`, `Anzahl Einzelteil`, `ArbeitsschrittgruppeNr`, `Montagezeit`) VALUES ('231', '600003', '6009', '1', '13', '20');
INSERT INTO `Arbeitsschritt` (`SchrittNr`, `MerkmalNr`, `EinzelteilNr`, `An